In [112]:
'''
将产成品仓库的存货编码转为销售部的存货编码
'''

import pandas as pd
import numpy as np
import re
import openpyxl
import easygui
import os


#仓库销售编码字典
fname_dic =  r"F:\a00nutstore\008\zw08\用友报价\仓库销售编码字典.xlsx"
df_dic = pd.read_excel(fname_dic)
dic_ck_xs = dict(zip(df0['仓库货号'],df0['销售货号']))
dic_xs_ck = dict(zip(df0['销售货号'],df0['仓库货号']))







In [114]:
fname_xs = r"F:\a00nutstore\008\zw08\用友报价\销售仓库价格库存1.xlsx"
# 销售编码
def xiaoshou(fname_xs):
    df_xs = pd.read_excel(fname_xs)
    df_xs = df_xs[~df_xs['类别'].isna()]
    return df_xs
df_xs = xiaoshou(fname_xs)

xs_lst = [
    '类别',
    '货号',
    '品名',
    '销售部存货大类编码',
    '含量',
    '汉办',
    '汉口北电商',
    '外地',
    '规格型号',
    '存货代码',
    '仓库货号'
]
df_xs = df_xs[xs_lst]


In [115]:
#产成品仓库库存
def chuliChangku(fname,sheet_name):
    df = pd.read_excel(fname,sheet_name = sheet_name)
    #删除第一列的空行
    df = df[~df['类别'].isna()]
    df = df[~df['类别'].str.contains('小计|累计|合   计')]
    df = df[~(abs(df['本日数量'] - 0) < 0.00001)]
    return df
fname_canchengpin = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\成品仓库7.24数据.xls"
sheet_name  =  0
df_canchengpin = chuliChangku(fname_canchengpin,sheet_name)

def addXiaoshouhuohao(df,dic_ck_xs):
    df_ck = df.assign(销售货号 = df['货号'].map(dic_ck_xs))
    # gp = df_ck.groupby('货号')
    gp = df_ck.groupby('货号')      #按销售货㞻汇总，使数据唯一性，为下一步合并做好准备
    df_ck = pd.DataFrame(gp.sum())
    df_ck = df_ck[['销售货号','本日数量']]
    df_ck = df_ck.reset_index()
    return df_ck
df_ck0 = addXiaoshouhuohao(df_canchengpin,dic_ck_xs)  #仓库
df_ck = df_ck0[['本日数量','销售货号','货号']]
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck_newAdd = df_ck.loc[df_t['销售货号'].isin([0,'0'])]
df_ck['销售货号'] = np.where(df_t['销售货号'].isin([0,'0']),df_ck['仓库货号'],df_ck['销售货号'] )
df_ck.to_excel('仓库库存.xlsx',index = False)
df_ck_newAdd = df_ck_newAdd[['仓库货号','销售货号']]
df_dic = pd.concat([df_dic,df_ck_newAdd])
df_dic.to_excel(fname_dic,index = False)

df_ck_newAdd.to_excel('仓库库存新增.xlsx',index = False)






C:\Users\redda\AppData\Local\Temp\ipykernel_29772\4141213283.py:26: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_ck.to_excel('仓库库存.xlsx',index = False)
C:\Users\redda\AppData\Local\Temp\ipykernel_29772\4141213283.py:29: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_dic.to_excel(fname_dic,index = False)
C:\Users\redda\AppData\Local\Temp\ipykernel_29772\4141213283.py:31: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_ck_newAdd.to_excel('仓库库存新增.xlsx',index = False)


In [116]:
def weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck):
    merge = pd.merge(df_xs, df_ck, left_on='货号', right_on='销售货号', how='right')
    merge['本日数量本'] = merge['本日数量']
    merge['本日数量件'] = merge['本日数量'] / merge['含量']
    del merge['本日数量']
    del merge['销售货号']
    gp = merge.groupby('货号')
    dic_agg = {'含量': 'mean', '汉办': 'mean', '汉口北电商': 'mean', '外地': 'mean', '本日数量本': 'sum'}
    right = gp.agg(dic_agg)
    right = right.reset_index()
    # right = right.rename(columns = {"销售货号":'货号'})
    left = df_xs[[
        '类别',
        '货号',
        '品名',
        '销售部存货大类编码',
         '规格型号',
    '存货代码'
        
    ]]
    mg = pd.merge(left, right, left_on='货号', right_on='货号',how = 'right')
    mg['本日数量件'] = mg['本日数量本'] / mg['含量']
    mg = mg.assign(仓库货号=mg['货号'].map(dic_xs_ck))
    return mg


result_beforeNew = weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck)
result_beforeNew.to_excel('销售仓库价格库存0724正-2.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_29772\3409069437.py:28: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result_beforeNew.to_excel('销售仓库价格库存0724正-2.xlsx',index = False)


In [ ]:
#7月16-7月25日新系统累计出库
lst = [
'仓库编码',
    '仓库',   
    '存货分类 (1级)',
 '存货分类 (2级)',
 '存货分类 (3级)',
 '存货分类 (4级)',
 '存货分类 (5级)',
 '存货编码',
 '存货',
 '存货代码',
  '数量(主单位)',
 '平均单价',
 '金额',
 '数量(辅单位)',
 '计量单位组合',
 '数量(主单位).1',
 '平均单价.1',
 '金额.1',
 '数量(辅单位).1',
 '计量单位组合.1',
 '数量(主单位).2',
 '平均单价.2',
 '金额.2',
 '数量(辅单位).2',
 '计量单位组合.2',
 '数量(主单位).3',
 '平均单价.3',
 '金额.3',
 '辅数量',
 '计量单位组合.3']
lst2 = [
    '仓库编码',
    '仓库',  
'class01',
 'class02',
 'class03',
 'class04',
 'class05',
 'code',
 'stock',
 'content',
  'begin_ben',
 '平均单价',
 '金额',
 'begin_jian',
 '计量单位组合',
 'ruku_ben',
 '平均单价.1',
 '金额.1',
 'ruku_jian',
 '计量单位组合.1',
 'chuku_ben',
 '平均单价.2',
 '金额.2',
 'chuku_jian',
 '计量单位组合.2',
 'end_ben',
 '平均单价.3',
 '金额.3',
 'end_jian',
 '计量单位组合.3']

lst3 =  [
 'class02',
 'class05',
 'code',
 'stock',
 'content',
 'begin_jian',
 'ruku_jian',
    'chuku_ben',
 'chuku_jian',
 'end_jian',
]




def chuli(fname):
    df =  pd.read_excel(fname,skiprows = 7)
    df = df.iloc[:,1:]
    df.columns = lst
    df.columns = lst2
    df['content'] = df['end_ben']/df['end_jian']
    df = df[df['class01'] != '制表人:']
    df = df[df['class01'] != '合计：']
    df = df[df['class01'].notnull()]
    df = df[lst3]
    return df

fname = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表20240723.xlsx"
chuku = chuli(fname)

chuku_new = chuku[['code','begin_jian','ruku_jian','chuku_ben','chuku_jian']]
chuku_new = chuku_new.groupby('code').sum()
chuku_new = chuku_new.reset_index()
chuku_new

In [ ]:
chuku_new.to_excel('测试期间出库.xlsx',index = False)

In [ ]:
#期初本、期初件
fname0 = r"F:\a00nutstore\008\zw08\用友报价\销售部001库存20240716正.xlsx"
df0 = pd.read_excel(fname0)
df0 = df0[['货号','本日数量本','本日数量件']]
df0 = df0.rename(columns = {'本日数量本':'qichu_ben','本日数量件':'qichu_jian'})
df0

In [ ]:
result0 = pd.merge(result_beforeNew,chuku_new,left_on = '货号',right_on = 'code',how = 'left')
result1 = pd.merge(result0,df0,on = '货号',how = 'left')
result1

In [ ]:
result1['期末件'] = result1['本日数量件'] - result1['chuku_jian']
result1['期末本'] = result1['本日数量本'] - result1['chuku_ben']
result1['变动件'] = result1['期末件'] - result1['qichu_jian']
result1['变动本'] = result1['期末本'] - result1['qichu_ben']

In [ ]:
result1.to_excel('产成品变动.xlsx',index = False)


In [132]:
#本月新增产品，将含量
df_xsEmpty = df_xs.iloc[:0]    #留df架构
df_xsAdd = pd.concat([df_xsEmpty,df_ck_newAdd])
df_xsAdd['仓库货号']= df_xsAdd['仓库货号'].astype('str')
df_content = df_canchengpin[['类别','货号','品名','含量']]
df_content['货号']= df_content['货号'].astype('str')
df_New = pd.merge(df_xsAdd,df_content,left_on = '仓库货号',right_on = '货号',how = 'left')
df_New['含量_x'] = df_New['含量_y'] 
df_New['类别_x'] = df_New['类别_y'] 
df_New['品名_x'] = df_New['品名_y'] 
df_New

C:\Users\redda\AppData\Local\Temp\ipykernel_29772\3856011408.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_content['货号']= df_content['货号'].astype('str')


,类别_x,货号_x,品名_x,销售部存货大类编码,含量_x,汉办,汉口北电商,外地,规格型号,存货代码,仓库货号,销售货号,类别_y,货号_y,品名_y,含量_y


In [133]:
cols = ['类别_x',
 '货号_x',
 '品名_x',
 '销售部存货大类编码',
 '含量_x',
 '汉办',
 '汉口北电商',
 '外地',
 '规格型号',
 '存货代码',
 '仓库货号',
  '含量_y']
df_New = df_New[cols]

del df_New['含量_y'] 
cols = cols[:-1]
cols = [x.replace('_x','') for x in cols]
df_New.columns = cols
df_New.to_excel('新增产品.xlsx',index = False)

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地,规格型号,存货代码,仓库货号
